In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
import time
import urllib
from tqdm import tqdm
import sys
import os

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import re
import datetime
import json
from IPython.display import display

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from lxml.html import fromstring
from bs4 import BeautifulSoup

In [2]:
# 수집 날짜 지정(전일자)
before_one_day = datetime.datetime.now() - datetime.timedelta(days=1)
D, db_date = before_one_day.strftime("%Y-%m-%d"), before_one_day.strftime("%y%m%d")
print(D, db_date)

2023-11-02 231102


---

In [3]:
# 뉴스 링크 담을 DataFrame
news_href = pd.DataFrame(columns = ['news', 'title', 'href'])

drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

crawl = True
page_num = 1

drv.get(f'https://entertain.naver.com/now#sid=106&date={D}&page={page_num}')
time.sleep(1)

while crawl == True:
    soup = BeautifulSoup(drv.page_source, 'html.parser')

    list_news = soup.find('ul', {'class': 'news_lst news_lst2'})
    get_list = list_news.findAll('li')

    # 해당 페이지 뉴스 기관, 기사 제목, 기사 링크 수집
    for lenth in tqdm(range(len(get_list))):
        content = get_list[lenth].find('div', {'class':'tit_area'})

        news_name = content.find('span').text.split('\n')[0]
        title = content.find('a').text
        href = 'https://entertain.naver.com' + content.find('a')['href']

        data = pd.DataFrame({'news':[news_name], 'title':[title], 'href':[href]})
        news_href = pd.concat([news_href, data], ignore_index=True)
            
    # 페이지 이동
    page = drv.find_element(By.CLASS_NAME, 'pg_num_area.newsest').find_elements(By.CLASS_NAME, 'pg_num')
    if int(page[-1].text) % 10 == 0:
        page_num += 1
        drv.get(f'https://entertain.naver.com/now#sid=106&date={D}&page={page_num}')
    else:
        if int(page[-1].text) > page_num:
            page_num += 1
            drv.get(f'https://entertain.naver.com/now#sid=106&date={D}&page={page_num}')
        else:
            crawl = False
            break
    time.sleep(2.5)
        
drv.close()

In [100]:
news_href

,news,title,href
0,TV리포트,"20년 미스터리 풀렸다.. 새벽2시 日서 사라진 김국진 ""강수지 만났다""('갓경규')",https://entertain.naver.com/now/read?oid=213&a...
1,일간스포츠,[TVis] 딘딘 “’세상에 이런 일이’ MC 예상 못해…’포스트 임성훈’ 고려한 ...,https://entertain.naver.com/now/read?oid=241&a...
2,스포츠조선,"""계속 토익강사하지"" 17기 옥순, 영철과 대화 중 눈물 ""연애 기회 없어"" ('나...",https://entertain.naver.com/now/read?oid=076&a...
3,OSEN,'고딩엄빠4' 극단적 선택 시도→극적 화해···엄마+딸 18세 임신 사연[종합],https://entertain.naver.com/now/read?oid=109&a...
4,마이데일리,"열연이 섬세하다…전종서, '발레리나'로 도약했다",https://entertain.naver.com/now/read?oid=117&a...
...,...,...,...
5150,뉴시스,"""꼬마와 아줌마 경호""…남현희·전청조, 명품 매장 목격담",https://entertain.naver.com/now/read?oid=003&a...
5151,마이데일리,"김정민, 파격 노출… 섹시 초콜릿 복근에 '입이 쩍' [MD★스타]",https://entertain.naver.com/now/read?oid=117&a...
5152,TV리포트,"박은빈이 직접 꼽은 '무인도의 디바' 관전포인트 ""속 시끄럽지 않게 할 자신 있어""",https://entertain.naver.com/now/read?oid=213&a...
5153,OSEN,'30일' 강하늘♥정소민 실제 연인같네..200만 앞두고 비하인드 대방출,https://entertain.naver.com/now/read?oid=109&a...


In [ ]:
news_href.to_excel(f'news_href_{db_date}.xlsx', index=False)

In [4]:
user_nm = db_id
passwd = db_pw
host_url = host_url
port_num = port_num
db_name = db_name
engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
engine_conn = engine.connect()

# insert
news_href.to_sql(f'table_name', con=engine, if_exists='append', index=None)
engine_conn.close()
engine.dispose()